In [1]:
import pandas as pd
import numpy as np

In [2]:
#dataset 읽기
df = pd.read_csv('data/lending-club-loan-data/loan.csv',sep=',')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.shape

(887379, 74)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 74 columns):
id                             887379 non-null int64
member_id                      887379 non-null int64
loan_amnt                      887379 non-null float64
funded_amnt                    887379 non-null float64
funded_amnt_inv                887379 non-null float64
term                           887379 non-null object
int_rate                       887379 non-null float64
installment                    887379 non-null float64
grade                          887379 non-null object
sub_grade                      887379 non-null object
emp_title                      835917 non-null object
emp_length                     842554 non-null object
home_ownership                 887379 non-null object
annual_inc                     887375 non-null float64
verification_status            887379 non-null object
issue_d                        887379 non-null object
loan_status          

In [5]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

In [6]:
#loan_status 컬럼의 값을 확인
df['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'],
      dtype=object)

In [7]:
df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [8]:
df['grade'].unique()

array(['B', 'C', 'A', 'E', 'F', 'D', 'G'], dtype=object)

In [9]:
df['grade'].value_counts()

B    254535
C    245860
A    148202
D    139542
E     70705
F     23046
G      5489
Name: grade, dtype: int64

In [10]:
df['purpose'].value_counts()

debt_consolidation    524215
credit_card           206182
home_improvement       51829
other                  42894
major_purchase         17277
small_business         10377
car                     8863
medical                 8540
moving                  5414
vacation                4736
house                   3707
wedding                 2347
renewable_energy         575
educational              423
Name: purpose, dtype: int64

In [11]:
df['title'].value_counts()

Debt consolidation                               414001
Credit card refinancing                          164331
Home improvement                                  40112
Other                                             31892
Debt Consolidation                                15760
Major purchase                                    12051
Business                                           6728
Medical expenses                                   6674
Car financing                                      5565
Consolidation                                      5381
debt consolidation                                 4834
Moving and relocation                              3921
Vacation                                           3805
Debt Consolidation Loan                            3804
Home buying                                        2365
Credit Card Consolidation                          2359
consolidation                                      2174
Personal Loan                                   

In [12]:
# grade(등급) 'A' 인 loan_amnt(대출금액)의 합계(sum)와 평균(mean)
df.loc[df['grade'] == 'A','loan_amnt'].sum()

2080587175.0

In [13]:
df.loc[df['grade'] == 'A','loan_amnt'].mean()

14038.86030552894

In [14]:
df.loc[df['grade'] == 'A','loan_amnt'].count()

148202

In [15]:
df.loc[df['grade'] == 'A','loan_amnt'].describe()

count    148202.000000
mean      14038.860306
std        7502.131394
min         500.000000
25%        8000.000000
50%       12000.000000
75%       20000.000000
max       35000.000000
Name: loan_amnt, dtype: float64

In [16]:
#df.loc[df['grade'] == 'A'].sum()

### 필요한 column 선택해서 새로운 DataFrame 생성하기
#### loan_amnt, loan_status, grade, int_rate, term
#### 대출금액,대출상태,등급,이자율,대출기간

In [17]:
df2=df[['loan_amnt','loan_status','grade','int_rate','term']]

In [20]:
df2.head()
# NaN 값을 포함한 데이터를 제거함
df2 = df2.dropna(how="any")

In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 887379 entries, 0 to 887378
Data columns (total 5 columns):
loan_amnt      887379 non-null float64
loan_status    887379 non-null object
grade          887379 non-null object
int_rate       887379 non-null float64
term           887379 non-null object
dtypes: float64(2), object(3)
memory usage: 40.6+ MB


In [22]:
df2.shape

(887379, 5)

### 1. 36개월, 60개월 대출총액 구하기

In [26]:
# 값을 저장할 dict 선언
term_by_loan_amnt_sum_dict = {}
#term 컬럼의 unique한 값을 가져오기
uniq_terms = df2['term'].unique()
for term in uniq_terms:
    loan_amnt_sum = df2.loc[df2['term'] == term,'loan_amnt'].sum()
    term_by_loan_amnt_sum_dict[term] = loan_amnt_sum

In [24]:
uniq_terms

array([' 36 months', ' 60 months'], dtype=object)

In [27]:
term_by_loan_amnt_sum_dict

{' 36 months': 7752507375.0, ' 60 months': 5341004575.0}

In [28]:
#결과가 저장된 dict를 Series 로 변환하기
term_by_loan_amnt_sum_obj = pd.Series(term_by_loan_amnt_sum_dict)

In [29]:
term_by_loan_amnt_sum_obj

 36 months    7.752507e+09
 60 months    5.341005e+09
dtype: float64

### 2. 각 loan_status(불량/우량)에 따른 대출 등급 분포 알아내기

In [31]:
total_status_catetory = df2['loan_status'].unique()

In [32]:
total_status_catetory

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'],
      dtype=object)

In [33]:
bad_status_category = total_status_catetory[[1,3,4,5,6,8]]

In [34]:
bad_status_category

array(['Charged Off', 'Default', 'Late (31-120 days)', 'In Grace Period',
       'Late (16-30 days)',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [49]:
#새로운 컬럼 생성 - loan_status가 bad_status_category에 
#포함되는지 체크해서 True/False값을 저장함
df2['bad_loan_status'] = df2['loan_status'].isin(bad_status_category)

In [50]:
df2.head()

,loan_amnt,loan_status,grade,int_rate,term,bad_loan_stauts,bad_loan_status
0,5000.0,Fully Paid,B,10.65,36 months,False,False
1,2500.0,Charged Off,C,15.27,60 months,True,True
2,2400.0,Fully Paid,C,15.96,36 months,False,False
3,10000.0,Fully Paid,C,13.49,36 months,False,False
4,3000.0,Current,B,12.69,60 months,False,False


In [44]:
df2.head()

,loan_amnt,loan_status,grade,int_rate,term,bad_loan_stauts
0,5000.0,Fully Paid,B,10.65,36 months,False
1,2500.0,Charged Off,C,15.27,60 months,True
2,2400.0,Fully Paid,C,15.96,36 months,False
3,10000.0,Fully Paid,C,13.49,36 months,False
4,3000.0,Current,B,12.69,60 months,False


In [51]:
df2.drop(columns=['bad_loan_stauts'],inplace=True)

In [52]:
# grade(등급)별 불량대출의 row 건수
df2.columns

Index(['loan_amnt', 'loan_status', 'grade', 'int_rate', 'term',
       'bad_loan_status'],
      dtype='object')

In [57]:
bad_loan_status_to_grades = df2.loc[df2['bad_loan_status'],'grade'].value_counts()

In [59]:
#index별 sorting
bad_loan_status_to_grades.sort_index()

A     3663
B    13456
C    19054
D    15859
E     9745
F     4383
G     1269
Name: grade, dtype: int64

In [60]:
#value 로 sorting (오름차순)
bad_loan_status_to_grades.sort_values()

G     1269
A     3663
F     4383
E     9745
B    13456
D    15859
C    19054
Name: grade, dtype: int64

In [61]:
#value 로 sorting (내림차순)
bad_loan_status_to_grades.sort_values(ascending=False)

C    19054
D    15859
B    13456
E     9745
F     4383
A     3663
G     1269
Name: grade, dtype: int64

In [63]:
#loan_status(대출상태)별 불량대출의 건수
df2.loc[df2['bad_loan_status'],'loan_status'].value_counts()

Charged Off                                            45248
Late (31-120 days)                                     11591
In Grace Period                                         6253
Late (16-30 days)                                       2357
Default                                                 1219
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64

In [64]:
df2.loc[~df2['bad_loan_status'],'loan_status'].value_counts()

Current                                               601779
Fully Paid                                            207723
Issued                                                  8460
Does not meet the credit policy. Status:Fully Paid      1988
Name: loan_status, dtype: int64

### 3. 대출금액과 이자율의 상관관계 구하기

In [65]:
df2['loan_amnt'].corr(df2['int_rate'])

0.14502309929883955

### 4. 등급별 대출금액의 건수를  파일로 저장하기

In [66]:
bad_loan_status_to_grades.to_csv('bad_loan_status_grade.csv',sep=',')

### 5. groupby 적용하기

In [69]:
#term(36개월, 60개월) 별로 합계 sum
#숫자 타입에 해당되는 모든 컬럼이 출력됨
df2.groupby('term').sum()

,loan_amnt,int_rate,bad_loan_status
term,,,
36 months,7.752507e+09,7.465102e+06,42931.0
60 months,5.341005e+09,4.289777e+06,24498.0


In [70]:
df2.groupby(df2['term']).sum()

,loan_amnt,int_rate,bad_loan_status
term,,,
36 months,7.752507e+09,7.465102e+06,42931.0
60 months,5.341005e+09,4.289777e+06,24498.0


In [71]:
#대출금액만 term별로 groupin 하려고 하면
df2['loan_amnt'].groupby(df2['term']).sum()

term
 36 months    7.752507e+09
 60 months    5.341005e+09
Name: loan_amnt, dtype: float64

In [72]:
df2['loan_amnt'].groupby(df2['term']).count()

term
 36 months    621125
 60 months    266254
Name: loan_amnt, dtype: int64

In [73]:
df2['loan_amnt'].groupby(df2['term']).mean()

term
 36 months    12481.396458
 60 months    20059.809712
Name: loan_amnt, dtype: float64

In [76]:
# grade별 대출금액을 counting
df2.groupby('grade').count()

grade
A    148202
B    254535
C    245860
D    139542
E     70705
F     23046
G      5489
Name: loan_amnt, dtype: int64

In [75]:
df2['loan_amnt'].groupby(df2['grade']).count()

grade
A    148202
B    254535
C    245860
D    139542
E     70705
F     23046
G      5489
Name: loan_amnt, dtype: int64

In [78]:
#불량 대출의 금액을 grade 별로 count()
df2.loc[df2['bad_loan_status'],'loan_amnt'].groupby(df2['grade']).count()

grade
A     3663
B    13456
C    19054
D    15859
E     9745
F     4383
G     1269
Name: loan_amnt, dtype: int64

In [81]:
#전체 대출금액을 loan_status 별로 counting
df2['loan_amnt'].groupby(df2['loan_status']).count().sort_values(ascending=
                                                                False)



loan_status
Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_amnt, dtype: int64

In [82]:
#불량 대출금액을 loan_status 별로 counting
df2.loc[df2['bad_loan_status'],'loan_amnt'].groupby(df2['loan_status']).count()

loan_status
Charged Off                                            45248
Default                                                 1219
Does not meet the credit policy. Status:Charged Off      761
In Grace Period                                         6253
Late (16-30 days)                                       2357
Late (31-120 days)                                     11591
Name: loan_amnt, dtype: int64

In [83]:
# bad_load_status 별, grade 별 couting
df2.groupby(['bad_loan_status','grade']).count()

loan_amnt  loan_status  int_rate    term
bad_loan_status grade                                          
False           A         144539       144539    144539  144539
                B         241079       241079    241079  241079
                C         226806       226806    226806  226806
                D         123683       123683    123683  123683
                E          60960        60960     60960   60960
                F          18663        18663     18663   18663
                G           4220         4220      4220    4220
True            A           3663         3663      3663    3663
                B          13456        13456     13456   13456
                C          19054        19054     19054   19054
                D          15859        15859     15859   15859
                E           9745         9745      9745    9745
                F           4383         4383      4383    4383
                G           1269         1269      1269    1269

In [84]:
df2.groupby(['bad_loan_status','grade'])['loan_amnt'].count()

bad_loan_status  grade
False            A        144539
                 B        241079
                 C        226806
                 D        123683
                 E         60960
                 F         18663
                 G          4220
True             A          3663
                 B         13456
                 C         19054
                 D         15859
                 E          9745
                 F          4383
                 G          1269
Name: loan_amnt, dtype: int64

In [85]:
df2.groupby(['bad_loan_status','grade'])[['loan_amnt','int_rate']].count()

loan_amnt  int_rate
bad_loan_status grade                     
False           A         144539    144539
                B         241079    241079
                C         226806    226806
                D         123683    123683
                E          60960     60960
                F          18663     18663
                G           4220      4220
True            A           3663      3663
                B          13456     13456
                C          19054     19054
                D          15859     15859
                E           9745      9745
                F           4383      4383
                G           1269      1269